In [478]:
import json
import pandas as pd
import numpy as np


In [479]:
# Change some pandas display options
pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_colwidth", 0)
pd.set_option("display.width", 2000)
pd.set_option("styler.format.precision", 5)


# Functions


In [480]:
def normalize_json(data):
    spaces_details = pd.json_normalize(data, record_path=["available_dates"], meta=["id"], errors="ignore")

    # Change columns type
    spaces_details["start"] = spaces_details["start"].astype("datetime64[ns]")
    spaces_details["end"] = spaces_details["end"].astype("datetime64[ns]")
    spaces_details["cancellable"] = spaces_details["cancellable"].astype(int)

    # Split date from time
    spaces_details["date"] = spaces_details["start"].dt.to_period("d").astype("datetime64[ns]")
    # spaces_details["date"] = spaces_details["date"].astype("str")

    return spaces_details


def add_cancellable_percent(spaces_details: pd.DataFrame):
    # Add new cancellable percentage column
    spaces_details["cancellable_percent"] = spaces_details["cancellable"] * spaces_details["total_time_span"]
    spaces_details["cancellable_percent"] /= spaces_details["total_time_span"].sum()

    # Drop columns
    spaces_details.drop(columns="cancellable", inplace=True)

    return spaces_details


def encode_time_span(spaces_details: pd.DataFrame):
    def encode(row: pd.Series):
        if row["end"].hour == 23:
            end = row["end"].hour
        else:
            end = row["end"].hour - 1
        start = row["start"].hour

        row.loc[start:end] = 1
        return row

    zeros_matrix = np.zeros(shape=(spaces_details.shape[0], 24), dtype="int")

    zeros_matrix = pd.DataFrame(zeros_matrix, columns=[i for i in range(24)])

    spaces_details = pd.concat([spaces_details, zeros_matrix], axis=1)

    return spaces_details.apply(encode, axis=1)


def add_total_time_span(spaces_details: pd.DataFrame):
    # Get available time span for each date
    # spaces_details["time_span"] = (spaces_details["end"] - spaces_details["start"]).astype("timedelta64[h]")
    spaces_details["total_time_span"] = spaces_details.loc[:, 0:23].sum(axis=1)

    # Correct time span to be 24 for whole day
    # spaces_details["time_span"] = spaces_details["time_span"].apply(lambda x: x + 1 if x == 23 else x)
    return spaces_details


# Read json data


In [481]:
with open("../data/raw/spaces_dummy_data_v3.json", mode="r") as f:
    data = json.load(f)


# Normalize data


In [482]:
spaces_details = normalize_json(data)
spaces_details = encode_time_span(spaces_details)
spaces_details = add_total_time_span(spaces_details)
spaces_details = add_cancellable_percent(spaces_details)


In [483]:
spaces_details.style.applymap(lambda x: 'background-color:#5fba7d' if x==1 else None)


,start,end,id,date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,total_time_span,cancellable_percent
0,2022-01-01 08:00:00,2022-01-01 16:00:00,A1,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,8,0.00000
1,2022-01-02 01:00:00,2022-01-02 02:00:00,A1,2022-01-02 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.05000
2,2022-01-03 01:00:00,2022-01-03 02:00:00,A1,2022-01-03 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.00000
3,2022-01-01 10:00:00,2022-01-01 12:00:00,A2,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0.10000
4,2022-01-02 01:00:00,2022-01-02 05:00:00,A2,2022-01-02 00:00:00,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0.20000
5,2022-01-01 12:00:00,2022-01-01 14:00:00,A3,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,2,0.00000
6,2022-01-01 14:00:00,2022-01-01 16:00:00,A4,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,2,0.10000


# First step

1. Get entire time span for each space
2. Check if one of them cover the intended request


In [484]:
individual_spaces = spaces_details.groupby(["id"]).sum()[["total_time_span", "cancellable_percent"]]
individual_spaces


,total_time_span,cancellable_percent
id,,
A1,10,0.05
A2,6,0.30
A3,2,0.00
A4,2,0.10


# Second step

- Create combinations from available spaces


In [485]:
spaces_ids = spaces_details["id"].unique()
# pd.MultiIndex.from_


In [486]:
spaces_details_group = spaces_details.groupby(["id", "date"]).sum()
unique_spaces = spaces_details_group.index.levels[0]
unique_dates = spaces_details_group.index.levels[1]


In [487]:
import itertools

combinations_lst = list(
    itertools.chain.from_iterable(itertools.combinations(unique_spaces, r) for r in range(2, len(unique_spaces) + 1))
)
combinations_lst


[('A1', 'A2'),
 ('A1', 'A3'),
 ('A1', 'A4'),
 ('A2', 'A3'),
 ('A2', 'A4'),
 ('A3', 'A4'),
 ('A1', 'A2', 'A3'),
 ('A1', 'A2', 'A4'),
 ('A1', 'A3', 'A4'),
 ('A2', 'A3', 'A4'),
 ('A1', 'A2', 'A3', 'A4')]

In [488]:
combinations_lst[4]


('A2', 'A4')

In [511]:
df = spaces_details_group.copy()


for comb in combinations_lst:
    dd = pd.DataFrame(spaces_details.query("id in @comb").loc[0:23].groupby("date").agg('sum'))
    
    dd[(dd>1)] = 1
    
    dd = pd.concat({"".join(comb): dd}, names=['id'])

    df = pd.concat([df, dd], axis=0)
    
df['total_time_span'] = df.loc[:, 0:23].sum(axis=1)
df.style.applymap(lambda x: 'color:darkorange' if x==1 else None, subset=list(range(0,24)))
df.reset_index(level=1, inplace=True)
df['date'] = df['date'].astype(str)
df.set_index('date', append=True, inplace=True)
df

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  total_time_span  cancellable_percent
id       date                                                                                                                                  
A1       2022-01-01  0  0  0  0  0  0  0  0  1  1  1   1   1   1   1   1   0   0   0   0   0   0   0   0   8                0.00               
         2022-01-02  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.05               
         2022-01-03  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.00               
A2       2022-01-01  0  0  0  0  0  0  0  0  0  0  1   1   0   0   0   0   0   0   0   0   0   0   0   0   2                0.10               
         2022-01-02  0  1  1  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4                0.20               
A3       2022-01-01  0  0  0  0  0  0  0  0  0  0  0   0   1   1   0   0   0   0   0   0   0   0   0   0   2                0.00               
A4       2022-01-01  0  0  0  0  0  0  0  0  0  0  0   0   0   0   1   1   0   0   0   0   0   0   0   0   2                0.10               
A1A2     2022-01-01  0  0  0  0  0  0  0  0  1  1  1   1   1   1   1   1   0   0   0   0   0   0   0   0   8                0.10               
         2022-01-02  0  1  1  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4                0.25               
         2022-01-03  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.00               
A1A3     2022-01-01  0  0  0  0  0  0  0  0  1  1  1   1   1   1   1   1   0   0   0   0   0   0   0   0   8                0.00               
         2022-01-02  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.05               
         2022-01-03  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.00               
A1A4     2022-01-01  0  0  0  0  0  0  0  0  1  1  1   1   1   1   1   1   0   0   0   0   0   0   0   0   8                0.10               
         2022-01-02  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.05               
         2022-01-03  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.00               
A2A3     2022-01-01  0  0  0  0  0  0  0  0  0  0  1   1   1   1   0   0   0   0   0   0   0   0   0   0   4                0.10               
         2022-01-02  0  1  1  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4                0.20               
A2A4     2022-01-01  0  0  0  0  0  0  0  0  0  0  1   1   0   0   1   1   0   0   0   0   0   0   0   0   4                0.20               
         2022-01-02  0  1  1  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4                0.20               
A3A4     2022-01-01  0  0  0  0  0  0  0  0  0  0  0   0   1   1   1   1   0   0   0   0   0   0   0   0   4                0.10               
A1A2A3   2022-01-01  0  0  0  0  0  0  0  0  1  1  1   1   1   1   1   1   0   0   0   0   0   0   0   0   8                0.10               
         2022-01-02  0  1  1  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4                0.25               
         2022-01-03  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1                0.00               
A1A2A4   2022-01-01  0  0  0  0  0  0  0  0  1  1  1   1   1   1   1   1   0   0   0   0   0   0   0   0   8                0.20               
         2022-01-02  0  1  1  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4                0.25     

In [513]:
# Convert multiindex dataframe to json
data = {level: df.xs(level).to_dict("index") for level in df.index.levels[0]}
# D = spaces_details_group.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict()).to_dict()

with open("./jsdf.json", mode="w") as f:
    json.dump(data, f, indent=4)



In [491]:
# Get total time span
spaces_details_group.groupby("id").sum()[["time_span"]].plot.bar(y="time_span")



KeyError: "None of [Index(['time_span'], dtype='object')] are in the [columns]"